<a href="https://colab.research.google.com/github/ujwalkr94/LLM_Developer/blob/main/notebooks/Firecrawl_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the requirements

In [ ]:
!pip install -q llama-index==0.12.12 openai==1.59.6 tiktoken==0.8.0 llama-index-readers-web==0.3.4 firecrawl-py==1.10.1

# (OR) To resolve the dependency issue.
# !pip uninstall -q torch torchvision torchaudio
# !pip install -q torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu126
# !pip install -q llama-index==0.12.12 openai==1.59.6 tiktoken==0.8.0 llama-index-readers-web==0.3.4 firecrawl-py==1.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 77.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### SET THE ENVIRONMENT VARIABLES

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"
FIRECRAWL_API_KEY = "<FIRECRAWL_API_KEY>"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY1')
# os.environ["FIRECRAWL_API_KEY"] = userdata.get('FIRECRAWL_API_KEY1')

# FIRECRAWL_API_KEY = userdata.get('FIRECRAWL_API_KEY1')

# SCRAPE WITH FIRECRAWL

## IMPORT THE FIRECRAWL WEBREADER

Firecrawl allows you to turn entire websites into LLM-ready markdown

Get the API key here
https://www.firecrawl.dev/app/api-keys

In [ ]:
from llama_index.readers.web import FireCrawlWebReader

In [ ]:

# using firecrawl to crawl a website
firecrawl_reader = FireCrawlWebReader(
    api_key=FIRECRAWL_API_KEY,  # Replace with your actual API key from https://www.firecrawl.dev/
    mode="scrape",
)

# Load documents from a single page URL
documents = firecrawl_reader.load_data(url="https://towardsai.net/")

In [ ]:
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter

# To increase chunk size globally
# Settings.chunk_size = 2048  # or even larger like 4096
# Settings.chunk_overlap = 200


# node parser with larger chunk size only for this index
node_parser = SentenceSplitter(
    chunk_size=2048,
    chunk_overlap=200,
)

index = VectorStoreIndex.from_documents(documents, transformations=[node_parser])
query_engine = index.as_query_engine()

In [ ]:
res = query_engine.query("What is towards AI aim?")

print(res.response)

print("-----------------")
# Show the retrieved nodes
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("URL\t", src.metadata['sourceURL'])
  print("Score\t", src.score)
  print("Description\t", src.metadata.get("description"))
  print("-_"*20)

Towards AI aims to be the leading AI community and content platform that makes AI accessible to all by providing high-quality publications, news, articles, and stories on AI and technology-related topics.
-----------------
Node ID	 b363bfbe-e859-4d62-9dac-8d85ce6613e3
Title	 Towards AI
URL	 https://towardsai.net/
Score	 0.8576130818181417
Description	 Towards AI is an online publication, which focuses on sharing high-quality publications, news, articles, and stories on AI and technology related topics., Louie's thoughts on the week's biggest AI developments. 
All major AI news, models, tools and papers covered. 
Read by over 130,000 AI Practitioners, Industry Professionals and Students. Click to read Towards AI Newsletter, a Substack publication.
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Node ID	 602747b8-4964-408d-a5fc-307e73c4c4e2
Title	 Towards AI
URL	 https://towardsai.net/
Score	 0.8493605511744174
Description	 Towards AI is an online publication, which focuses on sharing high-qual

# CRAWL A WEBSITE

## Load The CSV

CSV contains the list of tools and url of the page which we use to get information about the tool.

In [ ]:
import requests
import csv

# Google Sheets file URL (CSV export link)
url = 'https://docs.google.com/spreadsheets/d/1gHB-aQJGt9Nl3cyOP2GorAkBI_Us2AqkYnfqrmejStc/export?format=csv'

# Send a GET request to fetch the CSV file
response = requests.get(url)

response_list = []
# Check if the request was successful
if response.status_code == 200:
    # Decode the content to a string
    content = response.content.decode('utf-8')

    # Use the csv.DictReader to read the content as a dictionary
    csv_reader = csv.DictReader(content.splitlines(), delimiter=',')
    response_list = [row for row in csv_reader]
else:
    print(f"Failed to retrieve the file: {response.status_code}")


In [ ]:
import random

start_index = random.randint(0, len(response_list) - 3)
website_list = response_list[start_index:start_index+10] # Crawling 10 websites only.

In [ ]:
import pprint
print("CSV data")
pprint.pprint(website_list)

CSV data
[{'': '',
  'Category': 'Data Serialization',
  'Company': '',
  'Description': 'Data serialization system',
  'Is a direct URL company /tool website?': 'Yes',
  'Name': 'Apache Avro',
  'Parent': 'Apache',
  'Tool Type': 'Format/Algorithm/System',
  'URL': 'https://avro.apache.org/'},
 {'': '',
  'Category': 'Data Serialization',
  'Company': '',
  'Description': 'Insanely fast data interchange format and capability-based '
                 'RPC system',
  'Is a direct URL company /tool website?': 'Yes',
  'Name': "Cap'n Proto",
  'Parent': '',
  'Tool Type': 'Format/Algorithm/System',
  'URL': 'https://capnproto.org/'},
 {'': '',
  'Category': 'Data Storage',
  'Company': '',
  'Description': 'Fast, lightweight binary columnar data format',
  'Is a direct URL company /tool website?': 'No',
  'Name': 'Feather',
  'Parent': '',
  'Tool Type': 'Format/Algorithm/System',
  'URL': 'https://arrow.apache.org/docs/python/feather.html'},
 {'': '',
  'Category': 'Data Storage',
  'Com

## Initialize the Firecrawl

In [ ]:
import os
from firecrawl import FirecrawlApp
app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)

In [ ]:
import time

# Crawl websites and handle responses
url_response = {}
crawl_per_min = 1  # Max crawl per minute

# Track crawls
crawled_websites = 0
scraped_pages = 0

for i, website_dict in enumerate(website_list):
    url = website_dict.get('URL')
    print(f"Crawling: {url}")

    try:
        response = app.crawl_url(
            url,
            params={
                'limit': 5,  # Limit pages to scrape per site.
                'scrapeOptions': {'formats': ['markdown', 'html']}
            }
        )
        crawled_websites += 1

    except Exception as exc:
        print(f"Failed to fetch {url} -> {exc}")
        continue

    # Store the scraped data and associated info in the response dict
    url_response[url] = {
        "scraped_data": response.get("data"),
        "csv_data": website_dict
    }

    # Pause to comply with crawl per minute limit for free version its 1 crawl per minute
    if i!=len(website_list) and (i + 1) % crawl_per_min == 0:
        print("Pausing for 1 minute to comply with crawl limit...")
        time.sleep(60)  # Pause for 1 minute after every crawl


Crawling: https://avro.apache.org/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://capnproto.org/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://arrow.apache.org/docs/python/feather.html
Pausing for 1 minute to comply with crawl limit...
Crawling: https://www.neonscience.org/resources/learning-hub/tutorials/about-hdf5
Pausing for 1 minute to comply with crawl limit...
Crawling: https://msgpack.org/index.html
Pausing for 1 minute to comply with crawl limit...
Crawling: https://parquet.apache.org/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://protobuf.dev/overview/#:~:text=Protocol%20Buffers%20are%20a%20language,it%20generates%20native%20language%20bindings.
Pausing for 1 minute to comply with crawl limit...
Crawling: https://zarr.dev/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://dzone.com/articles/bayesian-optimization-and-hyperband-bohb-hyperpara#:~:text=Bayesian%20Optimization%20and%20Hyperband%2

## Create  llamaindex documents from the scraped content

In [ ]:
from llama_index.core import Document
documents = []

for _, scraped_content in url_response.items():
    csv_data = scraped_content.get("csv_data")
    scraped_results = scraped_content.get("scraped_data")

    for scraped_site_dict in scraped_results:
        for result in scraped_results:
            markdown_content = result.get("markdown")
            title = result.get("metadata").get("title")
            url = result.get("metadata").get("sourceURL")
            documents.append(
                Document(
                    text=markdown_content,
                    metadata={
                        "title": title,
                        "url": url,
                        "description": csv_data.get("Description"),
                        "category": csv_data.get("Category")
                    }
                )
            )


# Create The RAG Pipeline.

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter

llm = OpenAI(model="gpt-4o-mini")
embed_model = OpenAIEmbedding(model="text-embedding-3-large")
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=30)

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [ ]:
from IPython.display import Markdown, display
def display_response(response):
    display(Markdown(f"<b>{response}</b>"))

In [ ]:
query = "Explain Bayesian Optimization and Hyperband (BOHB) Hyperparameter Tuning" # Enter your query here, it should be relevant to the crawled websites
res = query_engine.query(query)
display_response(res)

print("-----------------")
# Show the retrieved nodes
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("URL\t", src.metadata['url'])
  print("Score\t", src.score)
  print("Description\t", src.metadata.get("description"))
  print("Category\t", src.metadata.get("category"))
  print("-_"*20)

<b>Bayesian Optimization and Hyperband (BOHB) is an advanced technique for hyperparameter tuning that combines two effective methodologies. 

Bayesian Optimization is a probabilistic model-based approach that utilizes a surrogate model, typically a Gaussian process, to represent the objective function, such as model accuracy. This method strategically explores the hyperparameter space, making informed decisions about where to sample next, which is particularly advantageous when evaluating the objective function is costly.

Hyperband, on the other hand, is a resource allocation strategy that optimally uses a limited budget—whether in terms of time or computational resources—to tune hyperparameters. It focuses on allocating resources to the most promising configurations while discarding those that perform poorly early in the process.

By integrating these two approaches, BOHB enhances the efficiency of hyperparameter optimization, allowing for effective exploration of large hyperparameter spaces, handling both continuous and categorical parameters. It automates the tuning process, minimizing the need for manual adjustments, and often achieves superior performance compared to traditional methods like grid search and random search.</b>

-----------------
Node ID	 59733826-963c-49e6-befc-9b5ac1c95819
Title	 BOHB Hyperparameter Tuning With an Example
URL	 https://dzone.com/articles/bayesian-optimization-and-hyperband-bohb-hyperpara#:~:text=Bayesian%20Optimization%20and%20Hyperband%20(BOHB)%20is%20a%20cutting%2Dedge,hyperparameters%20for%20machine%20learning%20models.
Score	 0.7274151454901601
Description	 Bayesian Optimization and HyperBand
Category	 Hyperparameter Tuning
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Node ID	 5a4bdf88-e521-42cf-adf5-6b0a02afdedd
Title	 BOHB Hyperparameter Tuning With an Example
URL	 https://dzone.com/articles/bayesian-optimization-and-hyperband-bohb-hyperpara#:~:text=Bayesian%20Optimization%20and%20Hyperband%20(BOHB)%20is%20a%20cutting%2Dedge,hyperparameters%20for%20machine%20learning%20models.
Score	 0.6926521645672065
Description	 Bayesian Optimization and HyperBand
Category	 Hyperparameter Tuning
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_


In [ ]:
# Querying not relevant to the crawled websites.

query = "What is qdrant?"
res = query_engine.query(query)
display_response(res)

print("-----------------")
# Show the retrieved nodes
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("URL\t", src.metadata['url'])
  print("Score\t", src.score)
  print("Description\t", src.metadata.get("description"))
  print("Category\t", src.metadata.get("category"))
  print("-_"*20)

<b>The provided information does not include details about Qdrant. It focuses on a format for the storage of chunked, compressed, N-dimensional arrays. For information about Qdrant, you may need to consult other sources.</b>

-----------------
Node ID	 ca003187-fb94-470f-966a-fed50025ab42
Title	 Zarr - Zarr
URL	 https://zarr.dev/
Score	 0.2227494278283149
Description	 Format for the storage of chunked, compressed, N-dimensional arrays
Category	 Data Storage
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
